In [ ]:
%%bash
mkdir final_b2_nobr2_w_buffers
mkdir final_b2_nobr2_w_buffers/v_or_h
mkdir final_b2_nobr2_w_buffers/v_or_h/remapped
mkdir final_b2_nobr2_w_buffers/v_or_h/remapped
mkdir final_b2_nobr2_w_buffers/v_or_h/setDifferences
mkdir final_b2_nobr2_w_buffers/final_output


In [ ]:
%%bash
prefix="./data_files/BiologicalReplicates"
awk '$3 - $2 <= $6 - $5 { OFS = "\t"; print $1, $2-5000, $3+5000, $4, $5, $6, $7 }' ${prefix}10000nt_MR10000000_ML50000_MD2000000_MW80000_WS2_SG2RH0.1_NSTR10_hg38_GM_b1t1_WholeGenome.txt > ./final_b2_nobr2_w_buffers/v_or_h/b1t1_5prime.txt
awk '$3 - $2 > $6 - $5 { OFS = "\t"; print $1, $2, $3, $4, $5-5000, $6+5000, $7 }' ${prefix}10000nt_MR10000000_ML50000_MD2000000_MW80000_WS2_SG2RH0.1_NSTR10_hg38_GM_b1t1_WholeGenome.txt > ./final_b2_nobr2_w_buffers/v_or_h/b1t1_3prime.txt

awk '$3 - $2 <= $6 - $5 { OFS = "\t"; print $1, $2-5000, $3+5000, $4, $5, $6, $7 }' ${prefix}10000nt_MR10000000_ML50000_MD2000000_MW80000_WS2_SG2RH0.1_NSTR10_hg38_GM_b2_WholeGenome.txt > ./final_b2_nobr2_w_buffers/v_or_h/b2_5prime.txt
awk '$3 - $2 > $6 - $5 { OFS = "\t"; print $1, $2, $3, $4, $5-5000, $6+5000, $7 }' ${prefix}10000nt_MR10000000_ML50000_MD2000000_MW80000_WS2_SG2RH0.1_NSTR10_hg38_GM_b2_WholeGenome.txt > ./final_b2_nobr2_w_buffers/v_or_h/b2_3prime.txt

awk '{ OFS = "\t"; print $4, $5, $6 }' ./final_b2_nobr2_w_buffers/v_or_h/b1t1_3prime.txt > ./final_b2_nobr2_w_buffers/v_or_h/b1t1_3prime_cut.txt
awk '{ OFS = "\t"; print $4, $5, $6 }' ./final_b2_nobr2_w_buffers/v_or_h/b2_3prime.txt > ./final_b2_nobr2_w_buffers/v_or_h/b2_3prime_cut.txt

bedtools intersect -v -a ./final_b2_nobr2_w_buffers/v_or_h/b2_3prime_cut.txt -b ./final_b2_nobr2_w_buffers/v_or_h/b1t1_3prime_cut.txt > ./final_b2_nobr2_w_buffers/v_or_h/setDifferences/b2_nob1t1_cut3prime.txt
bedtools intersect -v -a ./final_b2_nobr2_w_buffers/v_or_h/b2_5prime.txt -b ./final_b2_nobr2_w_buffers/v_or_h/b1t1_5prime.txt > ./final_b2_nobr2_w_buffers/v_or_h/setDifferences/b2_nob1t1_5prime.txt

bedtools intersect -v -a ./final_b2_nobr2_w_buffers/v_or_h/b1t1_3prime_cut.txt -b ./final_b2_nobr2_w_buffers/v_or_h/b2_3prime_cut.txt > ./final_b2_nobr2_w_buffers/v_or_h/setDifferences/b1t1_nob2_cut3prime.txt
bedtools intersect -v -a ./final_b2_nobr2_w_buffers/v_or_h/b1t1_5prime.txt -b ./final_b2_nobr2_w_buffers/v_or_h/b2_5prime.txt > ./final_b2_nobr2_w_buffers/v_or_h/setDifferences/b1t1_nob2_5prime.txt

In [ ]:
def read_intervals_by_chromosome(file_path):
    """
    Reads intervals from File A and organizes them in a dictionary keyed by chromosome.
    """
    chrom_intervals = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                chrom = parts[0]
#                 print(parts)
#                 print((parts[1], parts[2]))
#                 print((str(int(parts[1])-5000), str(int(parts[2])+5000)))
                interval = (str(int(parts[1])+5000), str(int(parts[2])-5000))
#                 interval = (parts[1], parts[2])  # (start, end) as strings
                if chrom not in chrom_intervals:
                    chrom_intervals[chrom] = set()
                chrom_intervals[chrom].add(interval)
    return chrom_intervals

def find_matching_loops_by_chromosome(file_a_path, file_b_path, output_path=None):
    """
    Finds and prints (or saves) loops from File B that match any interval from File A.
    Matching is chromosome-aware.
    """
    intervals_by_chr = read_intervals_by_chromosome(file_a_path)

    with open(file_b_path, 'r') as file_b:
        header = 1
        output_lines = []
        for line in file_b:
            parts = line.strip().split()
            if len(parts) >= 6:
                chrom1, start1, end1 = parts[0:3]
                chrom2, start2, end2 = parts[3:6]

#                 match1 = chrom1 in intervals_by_chr and (start1, end1) in intervals_by_chr[chrom1]
#                 match2 = chrom2 in intervals_by_chr and (start2, end2) in intervals_by_chr[chrom2]
                match1 = chrom1 in intervals_by_chr and (str(int(start1)+5000), str(int(end1)-5000)) in intervals_by_chr[chrom1]
                match2 = chrom2 in intervals_by_chr and (str(int(start2)+5000), str(int(end2)-5000)) in intervals_by_chr[chrom2]


                if match1 or match2:
                    c1,x1,x2,c2,y1,y2,pval = line.strip().split()
                    if match1:
                        x1 = str(int(x1)+5000)
                        x2 = str(int(x2)-5000)
                    else:
                        y1 = str(int(y1)+5000)
                        y2 = str(int(y2)-5000)      

                    print("\t".join([c1,x1,x2,c2,y1,y2,pval]))
                    output_lines.append("\t".join([c1,x1,x2,c2,y1,y2,pval])+"\n")

    # Output results
    if output_path:
        with open(output_path, 'w') as out_f:
            out_f.writelines(output_lines)
    else:
        for line in output_lines:
            print(line.strip())


In [ ]:
#remap 3' to file

pathA = "./final_b2_nobr2_w_buffers/v_or_h/b1t1_3prime.txt"
pathC = "./final_b2_nobr2_w_buffers/v_or_h/b2_3prime.txt"


stripepath={
    'br1': "./final_b2_nobr2_w_buffers/v_or_h/setDifferences/b1t1_nob2_cut3prime.txt",
    'br2_0': "./final_b2_nobr2_w_buffers/v_or_h/setDifferences/b2_nob1t1_cut3prime.txt",
}

find_matching_loops_by_chromosome(stripepath['br1'], pathA, "./final_b2_nobr2_w_buffers/v_or_h/remapped/"+"b1t1_uniq_3prime_remapped.txt")
find_matching_loops_by_chromosome(stripepath['br2_0'], pathC, "./final_b2_nobr2_w_buffers/v_or_h/remapped/"+"b2_uniq_3prime_remapped.txt")


In [ ]:
#remap 5' to file

pathA = "./final_b2_nobr2_w_buffers/v_or_h/b1t1_5prime.txt"
pathC = "./final_b2_nobr2_w_buffers/v_or_h/b2_5prime.txt"


stripepath={
    'br1': "./final_b2_nobr2_w_buffers/v_or_h/setDifferences/b1t1_nob2_5prime.txt",
    'br2_0': "./final_b2_nobr2_w_buffers/v_or_h/setDifferences/b2_nob1t1_5prime.txt",
}
find_matching_loops_by_chromosome(stripepath['br1'], pathA, "./final_b2_nobr2_w_buffers/v_or_h/remapped/"+"b1t1_nob2_5prime_remapped.txt")
find_matching_loops_by_chromosome(stripepath['br2_0'], pathC, "./final_b2_nobr2_w_buffers/v_or_h/remapped/"+"b2_nob1t1_5prime_remapped.txt")


In [ ]:
%%bash

cat ./final_b2_nobr2_w_buffers/v_or_h/remapped/b2_nob1t1_5prime_remapped.txt ./final_b2_nobr2_w_buffers/v_or_h/remapped/b2_uniq_3prime_remapped.txt > ./final_b2_nobr2_w_buffers/final_output/b2_uniq.txt
cat ./final_b2_nobr2_w_buffers/v_or_h/remapped/b1t1_nob2_5prime_remapped.txt ./final_b2_nobr2_w_buffers/v_or_h/remapped/b1t1_uniq_3prime_remapped.txt > ./final_b2_nobr2_w_buffers/final_output/b1t1_uniq.txt


In [ ]:
import os
import numpy as np


code_index = 9010202
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X']]


def APA(hic_file, loop_samples, chroms=chroms, resolution=10000, window=100):
    pile_sum = {
        sample: {ch: np.zeros((2 * window + 1, 2 * window + 1)) for ch in chroms} for sample in loop_samples
    }
    pile_count = {sample: {ch: 0 for ch in chroms} for sample in loop_samples}

    for ch in chroms:
        print('APA on', hic_file, ch)
        juicer = '/nfs/turbo/umms-drjieliu/juicer_tools_1.11.04_jcuda.0.8.jar'
        cmd = f'java -jar {juicer} dump oe KR {hic_file} {ch} {ch} BP {resolution} temp{code_index}.txt'
        print(cmd)
        os.system(cmd)

        max_size = 248956422 // resolution + 1
        mat = np.zeros((max_size, max_size))
        max_chr_size = 0
        if not os.path.exists(f'temp{code_index}.txt'):
            continue

        for line in open(f'temp{code_index}.txt'):
            try:
                lst = line.strip().split()
                if lst[2].lower() == 'nan':
                    continue
                p1, p2, v = int(lst[0]) // resolution, int(lst[1]) // resolution, float(lst[2])
                max_chr_size = max(max_chr_size, p1, p2)
                if p2 > max_size:
                    continue
                mat[p1, p2] += v
                if p1 != p2:
                    mat[p2, p1] += v
            except:
                pass
        os.remove(f'temp{code_index}.txt')




        if max_chr_size == 0 or np.sum(mat) == 0:
            continue

        mat = mat[:max_chr_size + 1, :max_chr_size + 1]

        for sample in loop_samples:
            print(' ', sample)
            f = open(loop_samples[sample])
            for line in f:
                if line.startswith('#'):
                    continue
                lst = line.strip().split()
                if lst[0] != ch:
                    continue
                p11, p12, p21, p22 = int(lst[1]), int(lst[2]), int(lst[4]), int(lst[5])
                if (p12 - p11) < (p22 - p21):
                    center = (p11 + p12) // 2 // resolution
                else:
                    center = (p21 + p22) // 2 // resolution

                if center < window or center >= max_chr_size - window:
                    continue
                sub_mat = mat[center-window: center+window+1, center-window: center+window+1]
                pile_sum[sample][ch] += sub_mat
                pile_count[sample][ch] += 1

    for sample in loop_samples:
        print('Final', sample)
        print(pile_count[sample])
        mat_sum, mat_count = np.zeros((2 * window + 1, 2 * window + 1)), 0
        for ch in chroms:
            mat_sum += pile_sum[sample][ch]
            mat_count += pile_count[sample][ch]
        piled_mat = mat_sum / mat_count

        np.save(f"./final_b2_nobr2_w_buffers/APA/uniq_all_alt_mat_{sample}.npy", piled_mat)




if __name__ == '__main__':
    #need GM12878 hic file: https://data.4dnucleome.org/files-processed/4DNFI1UEG1HD/
    hic_path = '/nfs/turbo/umms-drjieliu/proj/4dn/data/bulkHiC/GM12878/GM12878.hic'

    stripepath = {
        'b1t1': './final_b2_nobr2_w_buffers/final_output/b1t1_uniq.txt',
        'b2': './final_b2_nobr2_w_buffers/final_output/b2_uniq.txt',
    }


    APA(
        hic_file=hic_path, loop_samples=stripepath
    )


